In [0]:
!pip install sklearn
!pip install librosa
!pip install opencv-python
!pip install -U -q PyDrive
!apt install unzip

In [0]:
# only do this when on colab
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#will open another window to authorize; accept and copy-paste the code to here

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# access your google drive; you must right-click Prathik's folder and click 'add to my drive' 
#   to make a reference to it in your own drive (will not copy files and waste space)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore")
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#import librosa
import cv2 # opencv
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.models import load_model
from tqdm import tqdm
import gc
import os

Using TensorFlow backend.


In [0]:
# unpack the first folder for general debugging
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_01.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_02.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_03.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_04.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_05.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_06.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_07.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_08.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_09.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_10.zip?download=1'

In [0]:
# the rest of the files
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_11.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_12.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_13.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_14.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_15.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_16.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_17.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_18.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_19.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_20.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_21.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_22.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_23.zip?download=1'
!unzip '/content/drive/My Drive/Dataset/Video_Speech_Actor_24.zip?download=1'

In [0]:
# the CNN when we attempted to train from scratch
cnn = Sequential()
cnn.add(Conv2D(8, kernel_size=(3, 3),strides = (2,2),activation='relu',input_shape=(720, 1280, 3)))
cnn.add(Conv2D(8, (3, 3),strides = (2,2), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(4, 4),strides = (2,2)))
cnn.add(Dropout(0.5))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(9, activation='softmax')) # softmax bc more than 2 labels

cnn.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 359, 639, 8)       224       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 179, 319, 8)       584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 88, 158, 8)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 88, 158, 8)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 111232)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                3559456   
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)               

In [0]:
#emotion_y = ['N/A','Neutral','Calm','Happy','Sad','Angry','Fearful','Disgust','Surprise']

# the pre-trained CNN we used for transfer learning
cnn = load_model('drive/My Drive/trial_1.h5')
cnn.summary()

In [0]:
test_on = ['Actor_02','Actor_17','Actor_20']
j = 1
for actor in os.listdir('./'):
  if ('Actor' not in actor) or (actor in test_on):
    continue # skipping the extra folders
  path = os.path.join('./',actor)
  
  # progress tracking
  l = len(os.listdir(path))
  i = 0
  print('Training on',actor)
  j += 1

  for filename in os.listdir(path):
    path_inner = os.path.join(path, filename)
    file_emotion = [0]*9
    file_emotion[int(path_inner[17:19])] = 1

    cap = cv2.VideoCapture(path_inner)
    x = []
    y = []

    frame_count = 0
    while True:
      frame_count += 1
      # read the next frame from the file
      (grabbed, frame) = cap.read()

      # If the frame was not grabbed, then we have reached the end
      # of the stream
      if not grabbed:
        #print('Done,',i,'frames captured.')
        break
      # only grab each 5th frame to capture a slightly different expression each time; makes computation 5x faster
      elif frame_count % 5 == 0:
        x.append(frame/255)
        y.append(file_emotion)

    x = np.array(x)
    y = np.array(y)
    i += 1
    #print('    Training on',filename, int(i/l*100), '%')
    cnn.train_on_batch(x,y)
    del x
    del y
    gc.collect() # call garbage collector to prevent memory overflow
print('Done')

Training on Actor_22
Training on Actor_13
Training on Actor_04
Training on Actor_06
Training on Actor_19
Training on Actor_03
Training on Actor_10
Training on Actor_01
Training on Actor_23
Training on Actor_09
Training on Actor_07
Training on Actor_12
Training on Actor_16
Training on Actor_15
Training on Actor_14
Training on Actor_18
Training on Actor_24
Training on Actor_05
Training on Actor_08
Training on Actor_21
Done


In [0]:
cd drive/My Drive # navigate out of the colab notebook's temporary repository

/content/drive/My Drive


In [0]:
cnn.save('apm_cnn_full_res.h5')  # creates a HDF5 file after training the pictures at full resolution

In [0]:
cnn = load_model('apm_cnn_full_res.h5') # used this to recover CNN if colab reset without having to train the model again
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 359, 639, 8)       224       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 179, 319, 8)       584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 88, 158, 8)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 88, 158, 8)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 111232)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                3559456   
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)               

In [0]:
cd -

/content


In [0]:
train_on = ['Actor_0'+str(i) for i in range(1,10)] + ['Actor_'+str(i) for i in range(10, 25)]
train_on = [x for x in train_on if x not in test_on]
# set train/test split of actors

['Actor_02', 'Actor_17', 'Actor_20']

In [0]:
def testOnActor(actor):
  actor_dir = './'+actor+'/'
  loss = []
  acc = []
  #Test
  i = 0
  for filename in os.listdir(actor_dir):
    i += 1

    path = os.path.join(actor_dir, filename)
    file_emotion = [0]*9
    file_emotion[int(path[17:19])] = 1

    cap = cv2.VideoCapture(path)
    x = []
    y = []
    frame_count = 0
    while True:
      frame_count += 1
      # read the next frame from the file
      grabbed, frame = cap.read()

      # If the frame was not grabbed, then we have reached the end
      # of the stream
      if not grabbed:
        #print('Done,',i,'frames captured.')
        break
      elif frame_count % 5 == 0:
        x.append(frame/255)
        y.append(file_emotion)

    x = np.array(x)
    y = np.array(y)
    #print('Testing on',filename)
    test = cnn.test_on_batch(x,y)
    loss.append(test[0])
    acc.append(test[1])
    del x
    del y
    gc.collect()

  return loss,acc

In [0]:
loss23,acc23 = testOnActor(test_on[0])
print(test_on[0])
print('Average Loss =',np.array(loss23).mean())
print('Average Accuracy =',np.array(acc23).mean())

In [0]:
loss24,acc24 = testOnActor(train_on[2])
print(train_on[2])
print('Average Loss =',np.array(loss24).mean())
print('Average Accuracy =',np.array(acc24).mean())

Actor_04
Average Loss = 2.0313313
Average Accuracy = 0.19122635
